<a href="https://colab.research.google.com/github/rvernica/notebook/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install --upgrade duckdb
import duckdb
import pandas

from google.colab import userdata

In [24]:
con = duckdb.connect()
# Install extensions
con.install_extension("postgres")
con.install_extension("httpfs")
con.install_extension("ducklake")
# Create secrets for Postgres and Google Cloud Storage
con.execute("CREATE SECRET (TYPE gcs, KEY_ID ?, SECRET ?)", [userdata.get(k) for k in ("gcs_key_id", "gcs_secret")])
con.execute("CREATE SECRET (TYPE postgres, HOST ?, USER ?, PASSWORD ?)", [userdata.get(k) for k in ("postgres_host", "postgres_user", "postgres_password")])
# Connect to the data lake
postgres_dbname="ducklake"
bucket_name="side-lake"
con.execute(f"ATTACH 'ducklake:postgres:dbname={postgres_dbname}' AS lake (DATA_PATH 'gs://{bucket_name}/')");

In [25]:
con.table("lake.kaggle.titanic_train").df().head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [26]:
con.execute("DROP TABLE IF EXISTS train")
(con.table("lake.kaggle.titanic_train")
  .filter("Age IS NOT NULL")
  .filter("Embarked IS NOT NULL")
  .select("""
  PassengerId,
  Survived,
  IF(Pclass == 1, 1, 0) AS Pclass_1,
  IF(Pclass == 2, 1, 0) AS Pclass_2,
  IF(Sex == 'male', 1, 0) AS Male,
  Age/MAX(Age) OVER() AS Age_norm,
  SibSp,
  Parch,
  LOG(Fare + 1) AS Fase_lg,
  IF(Embarked == 'S', 1, 0) AS Embarked_S,
  IF(Embarked == 'C', 1, 0) AS Embarked_C,
  1 AS Bias""")
  .to_table("train"))
train = con.table("train")
n = train.count("*").fetchone()[0]
train.df().head()

,PassengerId,Survived,Pclass_1,Pclass_2,Male,Age_norm,SibSp,Parch,Fase_lg,Embarked_S,Embarked_C,Bias
0,1,0,0,0,1,0.2750,1,0,0.916454,1,0,1
1,2,1,1,0,0,0.4750,1,0,1.859038,0,1,1
2,3,1,0,0,0,0.3250,0,0,0.950608,1,0,1
3,4,1,1,0,0,0.4375,1,0,1.733197,1,0,1
4,5,0,0,0,1,0.4375,0,0,0.956649,1,0,1


In [27]:
print("Intialize training...")

con.execute("DROP TABLE IF EXISTS weights")
con.execute("DROP TABLE IF EXISTS forward")
con.execute("DROP TABLE IF EXISTS backward")

cols = pandas.DataFrame(train.columns[2:])
con.from_df(pandas.DataFrame(.0, index=[0], columns=train.columns[2:])).to_table("weights")
weights = con.table("weights")
forward = None
backward = None
rate = 0.1
iter = 1000

print(f"Start training for {iter} iterations...")
for i in range(iter):
    # --- Forward Pass ---
    pred_raw = ("weights." + cols[0] + " * train." + cols[0]).str.cat(sep=" + ")
    pred_sigmoid = f"1 / (1 + exp(-({pred_raw}))) as Pred"
    loss = f"-(Survived * log(Pred) + (1 - Survived) * log(1 - Pred)) AS Loss"
    forward_rel = train.cross(weights).select(f"PassengerId, {pred_sigmoid}, {loss}")
    if i == 0:
      forward_rel.to_table("forward")
      forward = con.table("forward")
    else:
      con.execute("TRUNCATE forward")
      forward_rel.insert_into("forward")

    # --- Backward Pass ---
    grad = ("SUM(" + cols[0] + f" * (Pred - Survived)) / {n} AS " + cols[0]).str.cat(sep=", ")
    backward_rel = train.join(forward, condition="PassengerId").aggregate(grad)
    if i == 0:
      backward_rel.to_table("backward")
      backward = con.table("backward")
    else:
      con.execute("TRUNCATE backward")
      backward_rel.insert_into("backward")

    # --- Weight Update ---
    update = ("weights." + cols[0] + f"- {rate} * backward." + cols[0] + " AS " + cols[0]).str.cat(sep=", ")
    weights.update(dict(zip(cols[0], weights.cross(backward).select(update).fetchone())))

    # --- Monitor Progress ---
    if (i + 1) % (iter // 10) == 0 or i == 0:
        cost = forward.aggregate(f"SUM(Loss) / {n} AS Cost").fetchone()[0]
        print(f"Iteration {i+1}/{iter}, Cost: {cost:.4f}")

print("Training finished.")
print("Final Weights:")
weights.df()

Intialize training...
Start training for 1000 iterations...
Iteration 1/1000, Cost: 0.3010
Iteration 100/1000, Cost: 0.2316
Iteration 200/1000, Cost: 0.2145
Iteration 300/1000, Cost: 0.2075
Iteration 400/1000, Cost: 0.2040
Iteration 500/1000, Cost: 0.2021
Iteration 600/1000, Cost: 0.2008
Iteration 700/1000, Cost: 0.2000
Iteration 800/1000, Cost: 0.1993
Iteration 900/1000, Cost: 0.1988
Iteration 1000/1000, Cost: 0.1984
Training finished.
Final Weights:


,Pclass_1,Pclass_2,Male,Age_norm,SibSp,Parch,Fase_lg,Embarked_S,Embarked_C,Bias
0,1.197878,0.747071,-2.436077,-0.917433,-0.310298,-0.114626,0.887683,-0.071328,0.357309,-0.069061
